In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy
import os
import sys

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Visualize point cloud

In [2]:
ply_point_cloud = o3d.data.PLYPointCloud()
print(ply_point_cloud.path)
pcd = o3d.io.read_point_cloud(ply_point_cloud.path)
print(np.asarray(pcd.points).shape)
o3d.visualization.draw_geometries([pcd], zoom=1,front=[0.4257, -0.2125, -0.8795], lookat=[2.6172, 2.0475, 1.532], up=[-0.0694, -0.9768, 0.2024])

C:\Users\nazib/open3d_data/extract/PLYPointCloud/fragment.ply
(196133, 3)


### Voxel Downsampling

In [3]:
downpcd = pcd.voxel_down_sample(voxel_size=0.01)
print(np.asarray(downpcd.points).shape)
o3d.visualization.draw_geometries([downpcd], zoom=1,front=[0.4257, -0.2125, -0.8795], lookat=[2.6172, 2.0475, 1.532], up=[-0.0694, -0.9768, 0.2024])

(94743, 3)


### Vertex normal estimation

In [4]:
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd], zoom=1,front=[0.4257, -0.2125, -0.8795], lookat=[2.6172, 2.0475, 1.532], up=[-0.0694, -0.9768, 0.2024], point_show_normal=True)

In [5]:
# Access estimated vertex normal.
print(downpcd.normals[100])

[-0.29541398  0.17810406 -0.93862108]


## Crop Point Cloud

In [6]:
demo_crop_data = o3d.data.DemoCropPointCloud() # Load the tutorial files
pcd = o3d.io.read_point_cloud(demo_crop_data.point_cloud_path) # Load the point cloud
vol = o3d.visualization.read_selection_polygon_volume(demo_crop_data.cropped_json_path) # Read the crop data
chair = vol.crop_point_cloud(pcd) # Crop the point cloud
o3d.visualization.draw_geometries([chair], zoom=1, front=[0.4257, -0.2125, -0.8795], lookat=[2.6172, 2.0475, 1.532], up=[-0.0694, -0.9768, 0.2024])

In [7]:
print(demo_crop_data.cropped_json_path)

C:\Users\nazib/open3d_data/extract/DemoCropPointCloud/cropped.json


## Paint point cloud

In [8]:
chair.paint_uniform_color([1, 0.706, 0])
o3d.visualization.draw_geometries([chair], zoom=1, front=[0.4257, -0.2125, -0.8795], lookat=[2.6172, 2.0475, 1.532], up=[-0.0694, -0.9768, 0.2024])

## Bounding Boxes

In [9]:
aabb = chair.get_axis_aligned_bounding_box()
aabb.color = (1, 0, 0)
obb = chair.get_oriented_bounding_box()
obb.color = (0, 1, 0)
o3d.visualization.draw_geometries([chair, aabb, obb], zoom=1, front=[0.4257, -0.2125, -0.8795], lookat=[2.6172, 2.0475, 1.532], up=[-0.0694, -0.9768, 0.2024])

## Convex Hull

In [10]:
# Load a mesh
bunny = o3d.data.BunnyMesh()
mesh = o3d.io.read_triangle_mesh(bunny.path)
mesh.compute_vertex_normals()

# Get sample the points with Poisson disk sampling
pcl = mesh.sample_points_poisson_disk(number_of_points=2000)

# Create hull from the Poisson disk sampled points
hull, _ = pcl.compute_convex_hull() # The raw hull. Enclosed mesh
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull) # Create a line set from the convex hull
hull_ls.paint_uniform_color((1, 0, 0))
o3d.visualization.draw_geometries([hull], zoom=1)

## DBSCAN Clustering

In [11]:
# Load point cloud
ply_point_cloud = o3d.data.PLYPointCloud()
pcd = o3d.io.read_point_cloud(ply_point_cloud.path)


with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd.cluster_dbscan(eps=0.02, min_points=20, print_progress=True))
    
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd], zoom=1, front=[0.4257, -0.2125, -0.8795], lookat=[2.6172, 2.0475, 1.532], up=[-0.0694, -0.9768, 0.2024])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 8
point cloud has 8 clusters


## Plane Segmentation

In [12]:
pcd_point_cloud = o3d.data.PCDPointCloud()
pcd = o3d.io.read_point_cloud(pcd_point_cloud.path)

# Segment plane
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)

[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

# Get inlier point cloud   
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud], zoom=1, front=[0.4257, -0.2125, -0.8795], lookat=[2.6172, 2.0475, 1.532], up=[-0.0694, -0.9768, 0.2024])

Plane equation: -0.05x + -0.10y + 0.99z + -1.07 = 0


## Planar Patch Detection

In [13]:
dataset = o3d.data.PCDPointCloud()
pcd = o3d.io.read_point_cloud(dataset.path)
assert (pcd.has_normals())

# using all defaults
oboxes = pcd.detect_planar_patches(
    normal_variance_threshold_deg=60,
    coplanarity_deg=75,
    outlier_ratio=0.75,
    min_plane_edge_length=0,
    min_num_points=0,
    search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))

print("Detected {} patches".format(len(oboxes)))

geometries = []
for obox in oboxes:
    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
    mesh.paint_uniform_color(obox.color)
    geometries.append(mesh)
    geometries.append(obox)
geometries.append(pcd)

o3d.visualization.draw_geometries(geometries,
                                  zoom=0.62,
                                  front=[0.4361, -0.2632, -0.8605],
                                  lookat=[2.4947, 1.7728, 1.5541],
                                  up=[-0.1726, -0.9630, 0.2071])

Detected 10 patches


## Planar Patch Detection